#Import Libraries

In [3]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

#Impot Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_test = pd.read_csv('/content/drive/MyDrive/projetos/Titanic - Machine Learning from Disaster/DADOS/test.csv')
df_train = pd.read_csv('/content/drive/MyDrive/projetos/Titanic - Machine Learning from Disaster/DADOS/train.csv')

In [5]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [6]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


#Preprocessing 

In [7]:
print(df_train['Sex'].value_counts())
print('------------------------------')
print(df_train['Embarked'].value_counts())
print('------------------------------')
print(df_train['Cabin'].value_counts())


male      577
female    314
Name: Sex, dtype: int64
------------------------------
S    644
C    168
Q     77
Name: Embarked, dtype: int64
------------------------------
G6             4
B96 B98        4
C23 C25 C27    4
F2             3
F33            3
              ..
B102           1
D28            1
E36            1
D6             1
A16            1
Name: Cabin, Length: 147, dtype: int64


In [8]:
sex_mapping = {
    'female' : 0,
    'male' : 1
}
embarked_mapping = {
    'S' : 0,
    'C' : 1,
    'Q' : 2,
}
df_train['Sex'] = df_train['Sex'].map(sex_mapping)
df_train['Embarked'] = df_train['Embarked'].map(embarked_mapping)
df_test['Sex'] = df_test['Sex'].map(sex_mapping)
df_test['Embarked'] = df_test['Embarked'].map(embarked_mapping)

In [9]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,2
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,0
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,2
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,0


In [10]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0.0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,0.0


In [11]:
df00 = df_train.drop(columns=['Name', 'Ticket', 'Cabin'])
df01 = df_test.drop(columns=['Name', 'Ticket', 'Cabin'])
dfN00 = df_train.drop(columns=[ 'Ticket', 'Cabin'])
dfN01 = df_test.drop(columns=[ 'Ticket', 'Cabin'])

In [12]:
df00.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,0.0
1,2,1,1,0,38.0,1,0,71.2833,1.0
2,3,1,3,0,26.0,0,0,7.9250,0.0
3,4,1,1,0,35.0,1,0,53.1000,0.0
4,5,0,3,1,35.0,0,0,8.0500,0.0


In [13]:
df01.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,7.8292,2
1,893,3,0,47.0,1,0,7.0000,0
2,894,2,1,62.0,0,0,9.6875,2
3,895,3,1,27.0,0,0,8.6625,0
4,896,3,0,22.0,1,1,12.2875,0


In [14]:
dfN00.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,7.2500,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,71.2833,1.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,7.9250,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,53.1000,0.0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,8.0500,0.0


In [15]:
dfN01.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,7.8292,2
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,7.0000,0
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,9.6875,2
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,8.6625,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,12.2875,0


In [16]:
dfN00['Name_contem_Condessa'] = dfN00['Name'].str.contains('Condessa').astype(int)
dfN00['Name_contem_Miss'] = dfN00['Name'].str.contains('Miss').astype(int)
dfN00['Name_contem_Mrs'] = dfN00['Name'].str.contains('Mrs').astype(int)
dfN00['Name_contem_Master'] = dfN00['Name'].str.contains('Master').astype(int)
dfN00['Name_contem_Col'] = dfN00['Name'].str.contains('Col').astype(int)
dfN00['Name_contem_Major'] = dfN00['Name'].str.contains('Major').astype(int)
dfN00['Name_contem_Mr'] = dfN00['Name'].str.contains('Mr').astype(int)

In [17]:
dfN01['Name_contem_Condessa'] = dfN01['Name'].str.contains('Condessa').astype(int)
dfN01['Name_contem_Miss'] = dfN01['Name'].str.contains('Miss').astype(int)
dfN01['Name_contem_Mrs'] = dfN01['Name'].str.contains('Mrs').astype(int)
dfN01['Name_contem_Master'] = dfN01['Name'].str.contains('Master').astype(int)
dfN01['Name_contem_Col'] = dfN01['Name'].str.contains('Col').astype(int)
dfN01['Name_contem_Major'] = dfN01['Name'].str.contains('Major').astype(int)
dfN01['Name_contem_Mr'] = dfN01['Name'].str.contains('Mr').astype(int)
dfN01 = dfN01.fillna(-1)

#Train

In [18]:
modelo = RandomForestClassifier(n_estimators=100 ,n_jobs=-1, random_state=0)


In [19]:
variaveis = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'] # 0.8093008739076156
variaveis02 = ['Sex', 'Age', 'Fare'] # 0.7845443196004993

variaveis03 = ['Pclass', 'Sex', 'Age', 'Fare'] # 0.8205118601747816

variaveis04 = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'] # 0.8115355805243446

variaveis05 = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Name_contem_Condessa', 'Name_contem_Miss', 'Name_contem_Mrs', 'Name_contem_Master', 'Name_contem_Col', 'Name_contem_Major', 'Name_contem_Mr'  ]
# 0.8160424469413234

In [20]:
X = df00[variaveis03].fillna(-1)
Y = df00['Survived'].fillna(-1)
Xn = dfN00[variaveis05].fillna(-1)


In [21]:
X_treino, X_valid, y_treino, y_valid = train_test_split(X, Y, test_size=0.3)

In [22]:

X_treino = X_treino.fillna(-1)
X_valid = X_valid.fillna(-1)



In [23]:
modelo.fit(X_treino, y_treino)
importances = modelo.feature_importances_
importances

array([0.08729245, 0.30060474, 0.26876724, 0.34333557])

In [25]:
prev = modelo.predict(X_valid)

In [26]:
modelo.score(X_valid, y_valid)

0.7985074626865671

## Cross Vali

In [27]:
scores = cross_val_score(modelo, Xn, Y, cv=10, scoring='accuracy' )
scores

array([0.72222222, 0.85393258, 0.74157303, 0.85393258, 0.85393258,
       0.88764045, 0.78651685, 0.7752809 , 0.85393258, 0.83146067])

In [28]:
scores.mean()

0.8160424469413234

#CSV

In [ ]:
csv00 = pd.Series(prevt, index = df_test['PassengerId'], name='Survived')
csv00.to_csv("teste00.csv", header=True)


In [ ]:
!head -n10 teste00.csv

PassengerId,Survived
892,0
893,0
894,0
895,0
896,0
897,0
898,1
899,0
900,1


#Others 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
clf = DecisionTreeClassifier()
scores02 = cross_val_score(clf, Xn, Y, cv=10, scoring='accuracy' )
scores02

array([0.7       , 0.80898876, 0.73033708, 0.84269663, 0.84269663,
       0.79775281, 0.82022472, 0.7752809 , 0.85393258, 0.84269663])

In [ ]:
scores02.mean()

0.8014606741573033

In [ ]:
clf_prev = clf.predict(X_valid)

In [ ]:
np.mean(y_valid == clf_prev)

0.7873134328358209

In [ ]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression()
scores03 = cross_val_score(reg, Xn, Y, cv=10, scoring='accuracy' )
scores03

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


array([0.83333333, 0.85393258, 0.76404494, 0.86516854, 0.82022472,
       0.7752809 , 0.79775281, 0.78651685, 0.87640449, 0.84269663])

In [ ]:
scores03.mean()

0.8215355805243446

In [ ]:
reg.fit(Xn, Y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
prevt01 = reg.predict(dfN01[variaveis05])
prevt01


array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [ ]:
csv01 = pd.Series(prevt01, index = df_test['PassengerId'], name='Survived')
csv01.to_csv("teste01.csv", header=True)

In [ ]:
!head -n10 teste01.csv

PassengerId,Survived
892,0
893,1
894,0
895,0
896,1
897,0
898,1
899,0
900,1
